# 0010_fastcore_meta_summary

## import

In [ ]:
from fastdebug.utils import *
from fastdebug.core import *

## fastcore

In [ ]:
import fastcore

In [ ]:
whichversion("fastcore")

fastcore: 1.5.27 
Python supercharged for fastai development    
Jeremy Howard and Sylvain Gugger 
https://github.com/fastai/fastcore/     
python_version: >=3.7     
/Users/Natsume/mambaforge/lib/python3.9/site-packages/fastcore


In [ ]:
whatinside(fastcore, lib=True)

The library has 21 modules
['_modidx',
 '_nbdev',
 'all',
 'basics',
 'dispatch',
 'docments',
 'docscrape',
 'foundation',
 'imports',
 'meta',
 'nb_imports',
 'net',
 'parallel',
 'script',
 'shutil',
 'style',
 'test',
 'transform',
 'utils',
 'xdg',
 'xtras']


In [ ]:
from fastcore.meta import *
import fastcore.meta as fm

In [ ]:
whatinside(fm, dun=True)

fastcore.meta has: 
13 items in its __all__, and 
43 user defined functions, 
19 classes or class objects, 
2 builtin funcs and methods, and
74 callables.

test_sig:            function    (f, b)
FixSigMeta:          metaclass, type    (name, bases, dict)
PrePostInitMeta:     metaclass, type    (name, bases, dict)
AutoInit:            class, PrePostInitMeta    (*args, **kwargs)
NewChkMeta:          metaclass, type    (name, bases, dict)
BypassNewMeta:       metaclass, type    (name, bases, dict)
empty2none:          function    (p)
anno_dict:           function    (f)
use_kwargs_dict:     decorator, function    (keep=False, **kwargs)
use_kwargs:          decorator, function    (names, keep=False)
delegates:           decorator, function    (to: function = None, keep=False, but: list = None)
method:              function    (f)
funcs_kwargs:        decorator, function    (as_method=False)


In [ ]:
whatinside(fm, func=True)

fastcore.meta has: 
13 items in its __all__, and 
43 user defined functions, 
19 classes or class objects, 
2 builtin funcs and methods, and
74 callables.

The user defined functions are:
_funcs_kwargs:       function    (cls, as_method)
_mk_param:           function    (n, d=None)
_rm_self:            function    (sig)
all_equal:           function    (a, b)
anno_dict:           function    (f)
any_is_instance:     function    (t, *args)
array_equal:         function    (a, b)
contextmanager:      decorator    (func)
copy:                function    (x)
delegates:           decorator    (to: function = None, keep=False, but: list = None)
df_equal:            function    (a, b)
empty2none:          function    (p)
equals:              function    (a, b)
funcs_kwargs:        decorator    (as_method=False)
in_colab:            function    ()
in_ipython:          function    ()
in_jupyter:          function    ()
in_notebook:         function    ()
ipython_shell:       function    ()
is_c

In [ ]:
whatinside(fm, clas=True)

fastcore.meta has: 
13 items in its __all__, and 
43 user defined functions, 
19 classes or class objects, 
2 builtin funcs and methods, and
74 callables.

The class objects are:
AutoInit:            class
BuiltinFunctionType: class
BuiltinMethodType:   class
BypassNewMeta:       metaclass
ExceptionExpected:   class
FixSigMeta:          metaclass
FunctionType:        class
MethodDescriptorType:class
MethodType:          class
MethodWrapperType:   class
NewChkMeta:          metaclass
NoneType:            class
Path:                class
PrePostInitMeta:     metaclass
SimpleNamespace:     class
WrapperDescriptorType:class
attrgetter:          class
itemgetter:          class
partial:             class


In [ ]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [ ]:
whatinside(fm, bltin=True)

fastcore.meta has: 
13 items in its __all__, and 
43 user defined functions, 
19 classes or class objects, 
2 builtin funcs and methods, and
74 callables.

The builtin functions or methods are:
['reduce', 'warn']
